In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

filepath = tf.keras.utils.get_file('shakespare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt' )
# ReadBinary from filepath, decode with utf-8, transform to lowercase
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

# select some parts of the text
text = text[300000:800000]


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:


characters = sorted( set(text) )
char_to_index = dict(( c, i) for i, c in enumerate(characters) )
index_to_char = dict( (i, c) for i, c in enumerate(characters) )

print(char_to_index)
print(index_to_char)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'a', 14: 'b', 15: 'c', 16: 'd', 17: 'e', 18: 'f', 19: 'g', 20: 'h', 21: 'i', 22: 'j', 23: 'k', 24: 'l', 25: 'm', 26: 'n', 27: 'o', 28: 'p', 29: 'q', 30: 'r', 31: 's', 32: 't', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z'}


In [3]:
SEQ_LENGTH = 40
STEP_SIZE= 3

# how ar
sentences = []
# complete e're you
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE ):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_characters.append(text[i + SEQ_LENGTH])

# create numpy array
x = np.zeros( (len(sentences), SEQ_LENGTH, len(characters)), dtype=bool )
y = np.zeros( (len(sentences), len(characters)), dtype=bool )

# Fill up the array with 2 for loops
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[ i, t, char_to_index[character] ] = 1
    y[i, char_to_index[next_characters[i]] ] = 1



In [4]:
# feed training data to neural network
model = Sequential()

# long short term memory will remember the input few iterations ago
# model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters)) ))

# Add an Input layer
model.add(InputLayer(shape=(SEQ_LENGTH, len(characters))))

# Add an LSTM layer with 128 neurons
model.add(LSTM(128))

model.add(Dense(len(characters)))
# predicting the best possible value by Adding an Activation layer with softmax
model.add(Activation('softmax'))

# learning_rate=0.01 - lr=0.01
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01) )

# Train the model
model.fit(x, y, batch_size=256, epochs=4 )

# Save the model textpoetgenerator.keras - LTS or textpoetgenerator.h5 - backward compatibility
model.save('textpoetgenerator.keras')

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 79s 118ms/step - loss: 2.5172
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 77s 118ms/step - loss: 1.8030
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 78s 120ms/step - loss: 1.6351
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 76s 117ms/step - loss: 1.5362


In [5]:
# Check if the model file is present using !ls or os.listdir('.') to avoid FileNotFoundError
# import os
# os.listdir('.')
model = tf.keras.models.load_model('textpoetgenerator.keras')

#Helper function from keras tutorial for preds=predictions
# takes model preds /high temp picks a more creative char.
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds) 
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH ]
    generated += sentence
    for i in range(length):
        x = np.zeros( (1, SEQ_LENGTH, len(characters)) )
        for t, character in enumerate(sentence):
            x[0, t, char_to_index[character] ] = 1
            
        prediction = model.predict(x, verbose=0)[0]
        next_index = sample(prediction, temperature )
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

print('----poet temperature=0.2-----')
print(generate_text(300, 0.2))

print('----poet temperature=0.4-----')
print(generate_text(300, 0.4))

print('----poet temperature=0.6-----')
print(generate_text(300, 0.6))

print('----poet temperature=0.8-----')
print(generate_text(300, 0.8))

print('----poet temperature=1.0-----')
print(generate_text(300, 1.0))

----poet temperature=0.2-----
remember me what a deal of world
i wander the friends the friends of the dreads
the world speak with the his loved and there is the sear
the traitor with strike the brother of the dreads,
and there is the friends and speak the friends
and streas the fiest the his soul that with thee?

polixenes:
as i have thee the earth the dream to thee.
----poet temperature=0.4-----

nurse:
even or odd, of all days in the sword,
to the doom at the loventions, and thou art
the world crown and the fither with the brother.

flice: of york:
why, a sonstreased with the world this will
i have the sons be the sonsure the hearts
and the mort me the friends and love the fair.

henry bolingbroke:
warwick, and when there nothin
----poet temperature=0.6-----
search, seek, and know how this foul murder,
for this like a villain shall time my frow die.

clown:
in beseech the toom your paris,
there nothing a desperant our songution,
the partilane, i have repartast the such doth doth wel